In [1]:
import numpy as np
import pandas as pd

# Problem Understanding

The National Health Service has determined that decisive action to treat diabetes is necessary. However, diagnosing it takes hours of doctors that are in high demand.<br>

Help them to predict who is diabetic and who is not based on data that non-medical stuff can obtain so that you reduce the number of people who:
- get treated without needing it
- don't get treated when they actually needed it

# Data Understanding

https://data.world/data-society/pima-indians-diabetes-database

In [2]:
df = pd.read_csv('pima-indians-diabetes.csv')
print(df.shape)
df.head()

(768, 9)


,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [3]:
X = df.drop('Outcome',axis=1)
y = df['Outcome']
X.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age
0,6,148,72,35,0,33.6,0.627,50
1,1,85,66,29,0,26.6,0.351,31
2,8,183,64,0,0,23.3,0.672,32
3,1,89,66,23,94,28.1,0.167,21
4,0,137,40,35,168,43.1,2.288,33


# Data preparation

### Split your X data in train and test datasets
Here is the documentation: https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html

In [4]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, random_state=42)

### Split your train data in train and validation datasets

In [5]:
from sklearn.model_selection import train_test_split

X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, train_size=0.8, random_state=42)

### Scale the 3 datasets using StandardScaler

In [6]:
from sklearn.preprocessing import StandardScaler 

ss = StandardScaler()
ss.fit(X_train, y_train)

X_train = ss.transform(X_train)
X_valid = ss.transform(X_valid)
X_test = ss.transform(X_test)

# Modelling and Model Evaluation

### Train a logistic regression model with NO regularisation

In [7]:
from sklearn.linear_model import LogisticRegression

model1 = LogisticRegression(penalty='none', solver='lbfgs')
model1.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='none',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

### Measure the accuracy and ROC_AUC of your model
Here is the documentation: https://scikit-learn.org/stable/modules/model_evaluation.html#classification-metrics

In [8]:
from sklearn.metrics import accuracy_score, roc_auc_score

mod1_train_pred = model1.predict(X_train)
mod1_train_auc = roc_auc_score(y_train, mod1_train_pred)
mod1_train_acc = accuracy_score(y_train, mod1_train_pred)

mod1_val_pred = model1.predict(X_valid)
mod1_val_auc = roc_auc_score(y_valid, mod1_val_pred)
mod1_val_acc = accuracy_score(y_valid, mod1_val_pred)

### Train a logistic regression model with L1 regularisation

In [9]:
model2 = LogisticRegression(C=1e12, penalty='l1', solver='liblinear')
model2.fit(X_train, y_train)

LogisticRegression(C=1000000000000.0, class_weight=None, dual=False,
                   fit_intercept=True, intercept_scaling=1, l1_ratio=None,
                   max_iter=100, multi_class='warn', n_jobs=None, penalty='l1',
                   random_state=None, solver='liblinear', tol=0.0001, verbose=0,
                   warm_start=False)

### Measure the accuracy and ROC_AUC of your model


In [10]:
mod2_train_pred = model2.predict(X_train)
mod2_train_auc = roc_auc_score(y_train, mod2_train_pred)
mod2_train_acc = accuracy_score(y_train, mod2_train_pred)

mod2_val_pred = model2.predict(X_valid)
mod2_val_auc = roc_auc_score(y_valid, mod2_val_pred)
mod2_val_acc = accuracy_score(y_valid, mod2_val_pred)

## Which model did you choose? Explain your reasoning

In [11]:
print('Without regularisation', '\n', f'AUC: {mod1_val_auc}  Accuracy: {mod1_val_acc}')
print('\n')
print('With regularisation', '\n', f'AUC: {mod2_val_auc}  Accuracy: {mod2_val_acc}')

Without regularisation 
 AUC: 0.730599647266314  Accuracy: 0.7886178861788617


With regularisation 
 AUC: 0.730599647266314  Accuracy: 0.7886178861788617


In [12]:
# The L1 regularisation doesnt seem to have made much of a difference with the models performances. 
# Not sure whcih I would pick, but here I will go with the 2nd model with regularisation

# Interprete your winning model

### Extract your intercept and coefficients

In [13]:
intec = model2.intercept_
coef = model2.coef_

### Show the difference in probabilities when changing the value of one of your predictors

In [14]:
coef_df = {'features': df.drop(columns='Outcome').columns, 'coefficients': coef[0]}
coef_df = pd.DataFrame(coef_df)
coef_df

,features,coefficients
0,Pregnancies,0.147133
1,Glucose,0.989553
2,BloodPressure,-0.345296
3,SkinThickness,0.135656
4,Insulin,-0.203526
5,BMI,0.801123
6,DiabetesPedigreeFunction,0.214327
7,Age,0.571571


In [15]:
# Glucose is the parameter that has the largest impact on the rate of change of our predicted output (or log of odds)
# This means that for every 1 unit increase in blood Glucose we will see......

### Extract the probability of a positive for your validation samples

In [16]:
uniq, counts = np.unique(mod2_val_pred, return_counts=True)
pos_prob = counts[1]/len(mod2_val_pred)

print(f'prediction: {uniq[0]}, counts:{counts[0]}')
print(f'prediction: {uniq[1]}, counts:{counts[1]}')

print(f'prob of a positive: {round( (pos_prob*100) , 2)}%')

prediction: 0, counts:93
prediction: 1, counts:30
prob of a positive: 24.39%


### Extract FPR, TPR and thresholds

In [17]:
from sklearn.metrics import  roc_curve

mod2_train_prob = model2.predict_proba(X_train)[:,1]
mod2_val_prob = model2.predict_proba(X_valid)[:,1]

fpr_train, tpr_train, thresh_train = roc_curve(y_train, mod2_train_prob)
fpr_val, tpr_val, thresh_val = roc_curve(y_valid, mod2_val_prob)

### Plot the ROC curves for your training, validation (and, if you are done, test) datasets

In [18]:
import matplotlib.pyplot as plt

plt.style.use('ggplot')
plt.figure(figsize=(11,7))

plt.plot(fpr_train, tpr_train, color='green', label='train')
plt.plot(fpr_val, tpr_val, color='yellow', label='validation')
plt.plot([0,1], [0,1], color='red', linestyle=':', label='50/50 guess')

plt.title('ROC')
plt.xlabel('FPR')
plt.ylabel('TPR')

plt.legend(loc=4)
plt.show()

<Figure size 1100x700 with 1 Axes>

# Threshold selection

### Estimate the prevalence in the environment where your model will be used

In [19]:
# Assuming the sample we have is representative of the study population the prevalence of the environment will be
# similar to the prevalence of our data

valu_count = df['Outcome'].value_counts()
prev = valu_count[1] / df.shape[0]

print(f'number of actual posotives: {valu_count[1]}')
print(f'total number of examples: {df.shape[0]}')
print(f'prevalence is {round(prev*100, 2)}%')

number of actual posotives: 268
total number of examples: 768
prevalence is 34.9%


### Estimate the costs for each unit of your FPs, TNs, FNs and TPs
**Hint:** You don't have data for this. You will have to do some research and deep thinking<br>
**Hint 2:** think of a £ value or something else that non-technical stakeholders could relate to<br>
**Hint 3:** They are going to be approximations and that's fine. Just create a good logic.<br>

In [20]:
# Assuming the monthly cost for each diabetic the public healthcare system has to care for is £1,000
# and the actual proportion of negative cases in the study population is identical to the proportion in our data

# FPc = (actual negative cases * FPR) * monthly cost



FPcost1 = 500*.2*1000     # No. of actual negatives are 500, a fpr of 20% and the montlhy cost of a diabetic patient
FPcost2 = 500*.4*1000     # No. of actual negatives are 500, a new fpr of 40% and the montlhy cost of a diabetic patient

# TNc = 
# FNc = 
# TPc =
print(f'monthly cost with a 20% FPR: £{FPcost1}')
print(f'monthly cost with a 40% FPR: £{FPcost2}')
print('So an increase in the rate of patients wronglfy identified as diabetic', 
      'could lead to a loss of 10\'s of thousands of £\'s')

monthly cost with a 20% FPR: £100000.0
monthly cost with a 40% FPR: £200000.0
So an increase in the rate of patients wronglfy identified as diabetic could lead to a loss of 10's of thousands of £'s


In [21]:
# Or like this...

# each diabetic patient costs £10,000 per cycle of care which lasts 6 months before re-testing for diabetes
# the cost of treating severe diabetic ailments like amputations, loss of sight etc are assumed to be very high £20,000

FPc = 10_000
TNc = 0
FNc = 20_000
TPc = 10_000

### Calculate your m parameter

In [22]:
term1 = (1-prev)/prev
term2 = (FPc - TNc) / (FNc - TPc)

m_param = term1*term2
m_param

1.865671641791045

### Calculate fm for each threshold

In [31]:
# fm = TPR - (m_param*FPR)
# fpr_val, tpr_val, thresh_val
# fm_df = pd.DataFrame.from_dict(thresh_scores, orient='index', columns=['thresholds', 'fm_scores'])

fm_list = []
for i, thresh in enumerate(thresh_val):
    fm_score = tpr_val[i] - (m_param * fpr_val[i])
    fm_list.append(fm_score)
    
fm_df = pd.DataFrame(index=list(range(0, len(thresh_val))))
fm_df['thresholds'] = thresh_val
fm_df['fm_scores'] = fm_list

fm_df

,thresholds,fm_scores
0,1.992922,0.000000
1,0.992922,0.023810
2,0.858124,0.142857
3,0.856481,0.119824
4,0.827549,0.167443
5,0.821594,0.144410
6,0.808551,0.192029
7,0.752234,0.145963
8,0.664729,0.288820
9,0.646546,0.265787


### Select the threshold with the highest fm score

In [41]:
max_fm = fm_df.max()[1]
fm_df.loc[fm_df['fm_scores'] == max_fm]

,thresholds,fm_scores
12,0.507128,0.409421


### Plot the confusion matrix for your selected threshold

In [42]:
from sklearn.metrics import confusion_matrix
conf_matrix = confusion_matrix(y_valid, mod2_val_pred)
conf_matrix

array([[74,  7],
       [19, 23]])

### Calculate your alpha / power / precision / accuracy
(Don't use any library for this exercise, in real life you can though)

In [44]:
# alpha = fp/ fp+tn
# power = tp/ tp+fn
# precision = tp/ tp+fp
# accuracy = tp+tn/ all actual target values
fp = conf_matrix[0][1]
tp = conf_matrix[1][1]
fn = conf_matrix[1][0]
tn = conf_matrix[0][0]

power = tp/(tp+fn)
precision = tp/(tp+fp)
print(f'The power is {power}', '\n', f'The precision is {precision}')

The power is 0.5476190476190477 
 The precision is 0.7666666666666667


### Explain in non-technical terms your alpha / power / precision / accuracy

In [ ]:
# 
# 
# 
# 
# 

# Actionable problem solving recommendations

### Write a brief paragraph telling your stakeholders something they can **DO** to be better off based on your model

In [ ]:
# 
# 
# 
# 
# 